In [7]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from datetime import datetime

# Parameters
get_img_size = (250, 250)
epoch_number = 120
loop_number = 1
n_splits = 10
splitter_test_size = 0.2
validation_size = 0.2
get_dropout_rate = 0.5
filter_size_1 = 10
filter_size_2 = 20
get_batch_size = 200
model_padding_size = 'same'
model_kernel_size = (5, 5)
optimizer_name = 'adam'
get_font = 'monospace'
x_distance = 0.72
y_distance = 0.30
transparency_level = 0.4
get_font_size = 9

# Directory Paths
directory_path = "Classified_Dataset/"
Folder_1 = os.path.join(directory_path, 'Depuy')
Folder_2 = os.path.join(directory_path, 'Tornier')

# Initialize images and labels
images = []
labels = []

# Helper function to load data
def getData(folder, label):
    file_names = os.listdir(folder)
    for file_name in file_names:
        path = os.path.join(folder, file_name)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, dsize=get_img_size)
            images.append(img)
            labels.append(label)

# Load data from both folders
getData(Folder_1, 0)  # Label 0 for Depuy
getData(Folder_2, 1)  # Label 1 for Tornier

print(f"Loaded {len(images)} images with corresponding labels.")



augmented_dir = 'Augmented_Tornier'
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)

# Augment Tornier images and save them
for batch in datagen.flow(tornier_images, batch_size=10, save_to_dir=augmented_dir, save_prefix='tornier', save_format='jpg'):
    if len(os.listdir(augmented_dir)) >= (294 - 71):  # Stop when balanced
        break


Loaded 365 images with corresponding labels.


In [8]:

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}  # Adjust according to your class labels
print("Class Weights:", class_weights_dict)

# Function to create the CNN model
def create_model():
    model = Sequential()
    model.add(Conv2D(filters=filter_size_1, kernel_size=model_kernel_size, padding=model_padding_size, input_shape=images.shape[1:] + (1,), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(get_dropout_rate))
    model.add(Conv2D(filters=filter_size_2, kernel_size=model_kernel_size, padding=model_padding_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(get_dropout_rate))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    return model

# Function to save performance summary
def save_performance_summary(epoch_number, sensitivity, specificity, accuracy_score):
    directory = "Excel_Files"
    filename = os.path.join(directory, "performance_summary.csv")
    if not os.path.exists(directory):
        os.makedirs(directory)
    if not os.path.exists(filename):
        with open(filename, 'w') as file:
            file.write("Epochs,Sensitivity,Specificity,Accuracy,Date_Time\n")
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filename, 'a') as file:
        file.write(f"{epoch_number},{sensitivity:.2f},{specificity:.2f},{accuracy_score:.2f},{current_time}\n")

# Stratified splitting
stratified_splitter = StratifiedShuffleSplit(n_splits=n_splits, test_size=splitter_test_size, random_state=42)

# Augment Tornier images to balance the dataset
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

tornier_images = np.array([img for img, label in zip(images, labels) if label == 1])
tornier_images = tornier_images.reshape(-1, *get_img_size, 1)

# Augment Tornier images and save them
for batch in datagen.flow(tornier_images, batch_size=10, save_to_dir='Augmented_Tornier', save_prefix='tornier', save_format='jpg'):
    if len(os.listdir('Augmented_Tornier')) >= (294 - 71):  # Stop when balanced
        break

# ShuffleSplit configuration for stratified splitting
for split_num, (train_idx, test_idx) in enumerate(stratified_splitter.split(images, labels)):
    print(f"\nRandom Split {split_num + 1}/{n_splits}: Running model training and evaluation")

    # Prepare training and testing datasets
    train_feature, test_feature = images[train_idx], images[test_idx]
    train_label, test_label = labels[train_idx], labels[test_idx]

    # Reshape and normalize
    train_feature_vector = train_feature.reshape(-1, *images.shape[1:]).astype('float32') / 255
    test_feature_vector = test_feature.reshape(-1, *images.shape[1:]).astype('float32') / 255
    train_label_onehot = to_categorical(train_label, num_classes=2)
    test_label_onehot = to_categorical(test_label, num_classes=2)

    # Create and compile the model
    model = create_model()
    model.compile(loss='binary_crossentropy', optimizer=optimizer_name, metrics=['accuracy'])

    # Train the model
    history = model.fit(train_feature_vector, train_label_onehot, 
                        validation_split=validation_size, 
                        epochs=epoch_number, 
                        batch_size=get_batch_size, verbose=0, 
                        class_weight=class_weights_dict)

    # Evaluate the model
    scores = model.evaluate(test_feature_vector, test_label_onehot)
    accuracy_score = scores[1]
    print(f"Test accuracy for split {split_num + 1}: {accuracy_score:.4f}")

    # Confusion matrix and metrics
    predicted_classes = np.argmax(model.predict(test_feature_vector), axis=1)
    cm = confusion_matrix(test_label, predicted_classes)
    print(f"Confusion Matrix for split {split_num + 1}:\n{cm}")

    TN, FP, FN, TP = cm.ravel()

    # Calculate sensitivity and specificity
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    print(f"Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")

    # Save metrics
    save_performance_summary(epoch_number, sensitivity, specificity, accuracy_score)

    # Plot training & validation accuracy and loss
    fig, ax1 = plt.subplots(2, 1, figsize=(10, 10))
    ax1[0].plot(history.history['accuracy'], label='Training Accuracy')
    ax1[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1[0].set_title(f'Model Accuracy :: Epochs :: {epoch_number}     Split No: {split_num + 1}')
    ax1[0].set_ylabel('Accuracy')
    ax1[0].set_xlabel('Epoch')
    ax1[0].legend(loc='upper left')

    ax1[1].plot(history.history['loss'], label='Training Loss')
    ax1[1].plot(history.history['val_loss'], label='Validation Loss')
    ax1[1].set_title(f'Model Loss :: Epochs :: {epoch_number}')
    ax1[1].set_ylabel('Loss')
    ax1[1].set_xlabel('Epoch')
    ax1[1].legend(loc='upper left')

    annotation_text = (
        f"Accuracy: {accuracy_score:.2f}\n"
        f"Sensitivity: {sensitivity:.2f}\n"
        f"Specificity: {specificity:.2f}\n"
    )

    ax1[0].text(x_distance, y_distance, annotation_text, transform=ax1[0].transAxes, fontsize=get_font_size, fontfamily=get_font, bbox=dict(facecolor='white', alpha=transparency_level))
    plt.tight_layout()
    plt.savefig(f'./results_{split_num + 1}.png')
    plt.close()

# End time
end_time = datetime.now()
print(f"Total Duration: {end_time - start_time}")


Class Weights: {0: 0.6207482993197279, 1: 2.5704225352112675}

Random Split 1/10: Running model training and evaluation


/home/ahmad/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-03 20:52:39.762621: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 500000000 exceeds 10% of free system memory.
2025-02-03 20:52:47.965610: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 500000000 exceeds 10% of free system memory.
2025-02-03 20:52:54.737539: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 500000000 exceeds 10% of free system memory.
2025-02-03 20:53:05.161242: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 500000000 exceeds 10% of free system memory.
2025-02-03 20:53:15.215921: W external/local_tsl/tsl

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 279ms/step - accuracy: 0.7312 - loss: 1.1494
Test accuracy for split 1: 0.7123
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 300ms/step
Confusion Matrix for split 1:
[[47 12]
 [ 9  5]]
Sensitivity: 0.3571, Specificity: 0.7966

Random Split 2/10: Running model training and evaluation


/home/ahmad/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 286ms/step - accuracy: 0.6168 - loss: 1.1144
Test accuracy for split 2: 0.6164
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 309ms/step
Confusion Matrix for split 2:
[[39 20]
 [ 8  6]]
Sensitivity: 0.4286, Specificity: 0.6610

Random Split 3/10: Running model training and evaluation


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 303ms/step - accuracy: 0.6217 - loss: 1.4152
Test accuracy for split 3: 0.6027
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 448ms/stepWARNING:tensorflow:6 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7605851fa7a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 312ms/step
Confusion Matrix for split 3:
[[39 20]
 [ 9  5]]
Sensitivity: 0.3571, Specificity: 0.6610

Ran

/home/ahmad/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.6628 - loss: 1.5607
Test accuracy for split 4: 0.6849
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 309ms/step
Confusion Matrix for split 4:
[[48 11]
 [12  2]]
Sensitivity: 0.1429, Specificity: 0.8136

Random Split 5/10: Running model training and evaluation


/home/ahmad/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 